[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_visual_signature.ipynb)

In [ ]:
%pip install pdftools_sdk
%pip install ipython

# Sign a PDF and add a visual appearance
Add a document signature with a visual appearance.
The visual appearance is configured using an XML or JSON
file, allowing the addition of text, images, or PDFs.

This signature consists of both a visible and a
non-visible part.
Only the non-visible part verifies the integrity of the
signed part of the document and authenticates the
signer's identity.
The signing certificate is read from a password-protected
PKCS#12 file (.pfx or .p12).

In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer
from pdftools_sdk.crypto.providers.built_in import Provider
from pdftools_sdk.sign import Appearance

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
certificate_file = 'INSERT-PFX-FILE'  # Placeholder for PFX file
password = 'INSERT-PASSWORD'
appearance_config_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/visual_appearance.xml'
appearance_config_file = 'visual_appearance.xml'
prepare_file(appearance_config_url, appearance_config_file)
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'SignedWithAppearance.pdf'

In [ ]:
def sign(certificate_file: str, password: str, appearance_config_file: str, input_path: str, output_path: str):
    # Create a session with the built-in cryptographic provider
    with Provider() as session:
        # Open certificate file
        with io.FileIO(certificate_file, 'rb') as pfx_stream:
            # Create signature configuration from PFX (or P12) file
            signature = session.create_signature_from_certificate(pfx_stream, password)

            # Create appearance from either an XML or JSON file
            with io.FileIO(appearance_config_file, 'rb') as appearance_stream:
                if appearance_config_file.endswith(".xml"):
                    signature.appearance = Appearance.create_from_xml(appearance_stream)
                else:
                    signature.appearance = Appearance.create_from_json(appearance_stream)

            signature.appearance.page_number = 1
            signature.appearance.custom_text_variables["company"] = "Daily Planet"

            # Open input document
            with io.FileIO(input_path, 'rb') as input_stream:
                with Document.open(input_stream) as input_document:
                    # Create stream for output file
                    with io.FileIO(output_path, 'wb+') as output_stream:
                        # Sign the input document
                        signer = Signer()
                        signer.sign(input_document, signature, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    # Sign a PDF document
    sign(certificate_file, password, appearance_config_file, input_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")